In [1]:
import dlib
import cv2
import numpy as np

# dlib의 얼굴 검출기와 얼굴 랜드마크 모델을 로드합니다.
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("data/shape_predictor_68_face_landmarks.dat")

# 입술 변화량 저장할 리스트 초기화
lip_changes = []

# 웹캠을 초기화합니다.
cap = cv2.VideoCapture(0)
frame_count = 0

while True:
    # 웹캠에서 프레임을 읽어옵니다.
    ret, frame = cap.read()

    # 얼굴 인식을 위해 흑백 이미지로 변환합니다.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 영역을 찾습니다.
    faces = detector(gray)

    if ret and frame_count % 5 == 0:  # 5프레임마다 출력
        for face in faces:
            # 얼굴 랜드마크를 예측합니다.
            landmarks = predictor(gray, face)

            # 입술 영역의 좌표 추출
        
            lip_center_point = np.array((landmarks.part(62).x, landmarks.part(62).y))

            # 이전 프레임의 포인트 초기화
            prev_point = lip_center_point

            # 입술 변화량 초기화
            lip_change = [0, 0, 0, 0]  # 48, 51, 54, 57 포인트의 변화량을 저장할 리스트

            # 입술 영역 표시
            for i in [48, 51, 54, 57]:
                x = landmarks.part(i).x - lip_center_point[0]
                y = landmarks.part(i).y - lip_center_point[1]

                # 포인트의 변화량 계산
                lip_change[i // 3 - 16] = int(np.sqrt(x ** 2 + y ** 2) ** 2)

            # 57 포인트 값이 220 이하인 경우에만 출력
            if lip_change[3] <= 210:
                lip_change = [0, 0, 0, 0]
                print(f"48 포인트 변화량 - {lip_change[0]}, 51 포인트 변화량 - {lip_change[1]}, 54 포인트 변화량 - {lip_change[2]}, 57 포인트 변화량 - {lip_change[3]}")
            else:
                print(f"48 포인트 변화량 - {lip_change[0]}, 51 포인트 변화량 - {lip_change[1]}, 54 포인트 변화량 - {lip_change[2]}, 57 포인트 변화량 - {lip_change[3]}")

            # 입술 변화량 리스트에 추가
            lip_changes.append(lip_change)

    # 결과를 화면에 표시합니다.
    cv2.imshow("Frame", frame)

    frame_count += 1

    # 'q' 키를 누르면 종료합니다.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 웹캠을 해제합니다.
cap.release()
cv2.destroyAllWindows()


48 포인트 변화량 - 1188, 51 포인트 변화량 - 122, 54 포인트 변화량 - 1124, 57 포인트 변화량 - 442
48 포인트 변화량 - 1170, 51 포인트 변화량 - 122, 54 포인트 변화량 - 1137, 57 포인트 변화량 - 442
48 포인트 변화량 - 1170, 51 포인트 변화량 - 122, 54 포인트 변화량 - 1192, 57 포인트 변화량 - 484
48 포인트 변화량 - 1255, 51 포인트 변화량 - 121, 54 포인트 변화량 - 1205, 57 포인트 변화량 - 530
48 포인트 변화량 - 1237, 51 포인트 변화량 - 122, 54 포인트 변화량 - 1192, 57 포인트 변화량 - 484
48 포인트 변화량 - 1325, 51 포인트 변화량 - 122, 54 포인트 변화량 - 1205, 57 포인트 변화량 - 484
48 포인트 변화량 - 1255, 51 포인트 변화량 - 122, 54 포인트 변화량 - 1205, 57 포인트 변화량 - 442
48 포인트 변화량 - 1255, 51 포인트 변화량 - 101, 54 포인트 변화량 - 1219, 57 포인트 변화량 - 442
48 포인트 변화량 - 1237, 51 포인트 변화량 - 121, 54 포인트 변화량 - 1274, 57 포인트 변화량 - 484
48 포인트 변화량 - 1153, 51 포인트 변화량 - 121, 54 포인트 변화량 - 1274, 57 포인트 변화량 - 484
48 포인트 변화량 - 1306, 51 포인트 변화량 - 121, 54 포인트 변화량 - 1274, 57 포인트 변화량 - 529


In [25]:
averages = []  # 평균값을 저장할 리스트

i = 0
while i < len(lip_changes):
    if lip_changes[i] != [0, 0, 0, 0]:  # 0이 아닌 지점 찾기
        start_idx = i
        end_idx = i
        while end_idx < len(lip_changes) and lip_changes[end_idx] != [0, 0, 0, 0]:  # 0이 되는 지점 찾기
            end_idx += 1

        # 0이 아닌 지점부터 0이 되는 지점까지의 각 포인트의 평균을 계산합니다.
        avg = [int(sum(x) / (end_idx - start_idx)) for x in zip(*lip_changes[start_idx:end_idx])]
        averages.append(avg)

        i = end_idx
    else:
        i += 1

# 결과 출력
for avg in averages:
    print(avg)

[680, 36, 487, 307]
[616, 40, 495, 302]
[625, 39, 522, 401]
[669, 36, 510, 388]
[533, 36, 415, 256]


In [26]:
averages

[[680, 36, 487, 307],
 [616, 40, 495, 302],
 [625, 39, 522, 401],
 [669, 36, 510, 388],
 [533, 36, 415, 256]]

In [27]:
import os

directory_path = "answer"

# 임계값 안에 들어오는 파일 이름들을 저장할 리스트
files_within_threshold = []

# 디렉토리 안의 모든 파일 순회
for filename in os.listdir(directory_path):
    if filename.endswith(".txt"):  # 확장자가 .txt인 파일만 처리
        file_path = os.path.join(directory_path, filename)

        # 파일 내용 읽어오기
        saved_lists = []
        with open(file_path, "r") as file:
            lines = file.readlines()
            for line in lines:
                line = line.strip()  # 줄 바꿈 문자 제거
                saved_list = list(map(int, line.split(",")))  # 쉼표로 분리하여 정수 리스트로 변환
                saved_lists.append(saved_list)

        # 파일 줄 수와 averages 리스트의 길이 비교
        if len(saved_lists) != len(averages):
            continue  # 비교하지 않고 다음 파일로 넘어감

        # 임계값 설정
        threshold = 300

        # 비교 수행
        is_in_threshold = True  # 임계값 안에 들어왔는지 여부
        for i in range(len(saved_lists)):
            for j in range(len(saved_lists[i])):
                if abs(saved_lists[i][j] - averages[i][j]) > threshold:
                    is_in_threshold = False
                    break
            if not is_in_threshold:
                break

        if is_in_threshold:
            files_within_threshold.append(os.path.splitext(filename)[0])  # .txt 확장자 제외한 파일 이름 저장

if files_within_threshold:
    print("상대방: ", end="")
    for filename in files_within_threshold:
        print(filename)
else:
    print("입모양과 맞는 단어가 없습니다.")


입모양과 맞는 단어가 없습니다.
